### Doc2Vec
Doc2vec is an adaptation of Word2Vec that allows us to learn document similarity. Doc2vec model by itself is an unsupervised method.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\rayha\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [25]:
data = pd.read_csv("data/Consumer_Complaints.csv", nrows=1000,usecols=['Consumer complaint narrative', 'Product']).dropna()
data.head()

,Product,Consumer complaint narrative
1,Student loan,When my loan was switched over to Navient i wa...
2,Credit card or prepaid card,I tried to sign up for a spending monitoring p...
7,Mortgage,"My mortgage is with BB & T Bank, recently I ha..."
13,Mortgage,The entire lending experience with Citizens Ba...
14,Credit reporting,My credit score has gone down XXXX points in t...


### Text preprocessing
Below we define a function to convert text to lower-case and strip punctuation/symbols from words and so on.

In [26]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text

In [27]:
data['Consumer complaint narrative'] = data['Consumer complaint narrative'].apply(cleanText)

In [28]:
data['Consumer complaint narrative'][1]

'when my loan was switched over to navient i was never told that i had a deliquint balance because with  i did not. when going to purchase a vehicle i discovered my credit score had been dropped from the  into the . i have been faithful at paying my student loan. i was told that navient was the company i had delinquency with. i contacted navient to resolve this issue you and kept being told to just contact the credit bureaus and epalin the situation and maybe they could help me. i was so angry that i just hurried and paid the balance off and then after tried to dispute the delinquency with the credit bureaus. i have had so much trouble bringing my credit score back up.'

In [29]:
##replace the column name
data.rename(columns = {'Consumer complaint narrative':'narrative'}, inplace = True)
data.head()

,Product,narrative
1,Student loan,when my loan was switched over to navient i wa...
2,Credit card or prepaid card,i tried to sign up for a spending monitoring p...
7,Mortgage,"my mortgage is with bb & t bank, recently i ha..."
13,Mortgage,the entire lending eperience with citizens ban...
14,Credit reporting,my credit score has gone down points in the l...


In [30]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [40]:
data_train = list(data["narrative"].head(10))
data_train[0]

'when my loan was switched over to navient i was never told that i had a deliquint balance because with  i did not. when going to purchase a vehicle i discovered my credit score had been dropped from the  into the . i have been faithful at paying my student loan. i was told that navient was the company i had delinquency with. i contacted navient to resolve this issue you and kept being told to just contact the credit bureaus and epalin the situation and maybe they could help me. i was so angry that i just hurried and paid the balance off and then after tried to dispute the delinquency with the credit bureaus. i have had so much trouble bringing my credit score back up.'

In [41]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_train)]

In [42]:
tagged_data

[TaggedDocument(words=['when', 'my', 'loan', 'was', 'switched', 'over', 'to', 'navient', 'i', 'was', 'never', 'told', 'that', 'i', 'had', 'a', 'deliquint', 'balance', 'because', 'with', 'i', 'did', 'not', '.', 'when', 'going', 'to', 'purchase', 'a', 'vehicle', 'i', 'discovered', 'my', 'credit', 'score', 'had', 'been', 'dropped', 'from', 'the', 'into', 'the', '.', 'i', 'have', 'been', 'faithful', 'at', 'paying', 'my', 'student', 'loan', '.', 'i', 'was', 'told', 'that', 'navient', 'was', 'the', 'company', 'i', 'had', 'delinquency', 'with', '.', 'i', 'contacted', 'navient', 'to', 'resolve', 'this', 'issue', 'you', 'and', 'kept', 'being', 'told', 'to', 'just', 'contact', 'the', 'credit', 'bureaus', 'and', 'epalin', 'the', 'situation', 'and', 'maybe', 'they', 'could', 'help', 'me', '.', 'i', 'was', 'so', 'angry', 'that', 'i', 'just', 'hurried', 'and', 'paid', 'the', 'balance', 'off', 'and', 'then', 'after', 'tried', 'to', 'dispute', 'the', 'delinquency', 'with', 'the', 'credit', 'bureaus', 

Here we have a list of four sentences as training data. Now I have tagged the data and its ready for training. Lets start training our model.

In [43]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

C:\Users\rayha\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


C:\Users\rayha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

**Note:** dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just uses the bag of words approach, which doesn’t preserve any word order.

So we have saved the model and it’s ready for implementation. Lets play with it.

In [44]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.docvecs['1'])

[('4', 0.6878265738487244), ('6', 0.6410451531410217), ('9', 0.578339695930481), ('5', 0.5374822616577148), ('3', 0.514997124671936), ('0', 0.49757710099220276), ('8', 0.46816176176071167), ('7', 0.3758920431137085), ('2', 0.31886008381843567)]
